In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [2]:
!kaggle datasets download -d marusagar/hand-gesture-detection-system

Dataset URL: https://www.kaggle.com/datasets/marusagar/hand-gesture-detection-system
License(s): other
 98% 1.57G/1.60G [00:12<00:00, 154MB/s]
100% 1.60G/1.60G [00:12<00:00, 137MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/hand-gesture-detection-system.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [21]:
import pandas as pd
train_df = pd.read_csv("/content/train.csv")
val_df = pd.read_csv("/content/val.csv")

In [22]:
train_df.head()

,WIN_20180925_17_08_43_Pro_Left_Swipe_new;Left_Swipe_new;0
0,WIN_20180925_17_18_28_Pro_Left_Swipe_new;Left_...
1,WIN_20180925_17_18_56_Pro_Left_Swipe_new;Left_...
2,WIN_20180925_17_19_51_Pro_Left_Swipe_new;Left_...
3,WIN_20180925_17_20_14_Pro_Left_Swipe_new;Left_...
4,WIN_20180925_17_21_28_Pro_Left_Swipe_new;Left_...


In [28]:
import pandas as pd

# Split the combined column into multiple columns
train_df = train_df.iloc[:, 0].str.split(';', expand=True)
val_df = val_df.iloc[:, 0].str.split(';', expand=True)

# Assign proper column names
train_df.columns = ['File_Name', 'Gesture_Type', 'Label']
val_df.columns = ['File_Name', 'Gesture_Type', 'Label']

train_df.head()

,File_Name,Gesture_Type,Label
0,WIN_20180925_17_18_28_Pro_Left_Swipe_new,Left_Swipe_new,0
1,WIN_20180925_17_18_56_Pro_Left_Swipe_new,Left_Swipe_new,0
2,WIN_20180925_17_19_51_Pro_Left_Swipe_new,Left_Swipe_new,0
3,WIN_20180925_17_20_14_Pro_Left_Swipe_new,Left_Swipe_new,0
4,WIN_20180925_17_21_28_Pro_Left_Swipe_new,Left_Swipe_new,0


In [29]:
import cv2
import numpy as np
import os
from tensorflow.keras.utils import to_categorical

In [31]:
IMG_SIZE = 64
SEQUENCE_LENGTH = 30
LABEL_MAP = {label: idx for idx, label in enumerate(train_df['Label'].unique())}


In [32]:
def load_sequence_images(base_path, folder_name):
    folder_path = os.path.join(base_path, folder_name)
    images = []
    for img_file in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0
        images.append(img)
    return np.array(images)

In [39]:
def create_dataset(df, base_path):
    X, y = [], []
    for _, row in df.iterrows():
        folder = row["File_Name"]
        label = row['Label']
        sequence = load_sequence_images(base_path, folder)
        if sequence.shape[0] == SEQUENCE_LENGTH:
            X.append(sequence)
            y.append(LABEL_MAP[label])
    return np.array(X), to_categorical(y, num_classes=len(LABEL_MAP))

X_train, y_train = create_dataset(train_df, '/content/train/train')
X_val, y_val = create_dataset(val_df, '/content/val/val')


In [40]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout


In [41]:
input_shape = (SEQUENCE_LENGTH, IMG_SIZE, IMG_SIZE, 3)

inputs = Input(shape=input_shape)
x = TimeDistributed(Conv2D(32, (3,3), activation='relu'))(inputs)
x = TimeDistributed(MaxPooling2D(2,2))(x)
x = TimeDistributed(Conv2D(64, (3,3), activation='relu'))(x)
x = TimeDistributed(MaxPooling2D(2,2))(x)
x = TimeDistributed(Flatten())(x)
x = LSTM(64)(x)
x = Dropout(0.5)(x)
outputs = Dense(len(LABEL_MAP), activation='softmax')(x)


In [42]:
model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30, 64, 64, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 30, 62, 62, 32) │           896 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 30, 31, 31, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 30, 29, 29, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 30, 14, 14, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 30, 12544)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │     3,227,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,247,621 (12.39 MB)

 Trainable params: 3,247,621 (12.39 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model.fit(X_train, y_train, epochs=20, batch_size=8, validation_data=(X_val, y_val))


Epoch 1/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 503s 6s/step - accuracy: 0.2587 - loss: 1.6851 - val_accuracy: 0.6061 - val_loss: 1.1874
Epoch 2/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 490s 6s/step - accuracy: 0.5934 - loss: 1.0868 - val_accuracy: 0.5657 - val_loss: 1.1021
Epoch 3/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 492s 6s/step - accuracy: 0.6602 - loss: 0.8783 - val_accuracy: 0.6566 - val_loss: 0.9355
Epoch 4/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 488s 6s/step - accuracy: 0.7842 - loss: 0.6205 - val_accuracy: 0.6566 - val_loss: 1.0374
Epoch 5/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 491s 6s/step - accuracy: 0.8409 - loss: 0.4882 - val_accuracy: 0.6364 - val_loss: 1.0722
Epoch 6/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 501s 6s/step - accuracy: 0.8779 - loss: 0.3608 - val_accuracy: 0.6869 - val_loss: 1.0021
Epoch 7/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 506s 6s/step - accuracy: 0.9322 - loss: 0.2311 - val_accuracy: 0.7172 - val_loss: 0.9558
Epoch 8/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 499s 6s/step - accuracy: 0.9779 - loss: 0.1152 - val_accuracy: 0.6465 - v

In [48]:
model.evaluate(X_val, y_val)

# Predict on one sample
pred = model.predict(np.expand_dims(X_val[54], axis=0))
predicted_class = list(LABEL_MAP.keys())[np.argmax(pred)]
print("Predicted:", predicted_class)


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.7008 - loss: 1.4866
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Predicted: 4


In [49]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)